In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from pathlib import Path
import re

In [3]:
# Batch Size
bs=16

In [4]:
folder_name = 'trashnet-images'
paths = Path(folder_name)

fnames = []
labels = []
label_pat = r'trashnet-images\/(.*)$'
for path in paths.ls():
    label = re.match(label_pat, str(path)).group(1)
    images = get_image_files(path)
    for image in images:
        fnames.append(image)
        labels.append(label)
np.random.seed(2)

data = ImageDataBunch.from_lists(
    paths, 
    fnames, 
    labels, 
    ds_tfms=get_transforms(), 
    valid_pct=0.2,
    size=224, 
    bs=bs).normalize(imagenet_stats)

In [5]:
print(data.classes)
len(data.classes),data.c

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


(6, 6)

# Fine Tune the model

In [6]:
learn = cnn_learner(data, models.resnet50, metrics=[accuracy])

In [7]:
#learn.lr_find()
#learn.recorder.plot()
learn.unfreeze()

In [8]:
learn.fit_one_cycle(7,
                    max_lr=slice(1e-6,1e-3),
                    callbacks=[SaveModelCallback(
                        learn, 
                        every='improvement', 
                        name='best-trashnet-model',
                        monitor='accuracy')])

epoch,train_loss,valid_loss,accuracy,time
0,1.033810,0.510908,0.833663,00:59
1,0.633031,0.437508,0.853465,00:56
2,0.466029,0.323483,0.897030,00:56
3,0.290737,0.330000,0.908911,00:56
4,0.198397,0.316943,0.918812,00:56
5,0.133996,0.299080,0.916832,00:56
6,0.136269,0.306481,0.922772,00:56


Better model found at epoch 0 with accuracy value: 0.8336633443832397.
Better model found at epoch 1 with accuracy value: 0.8534653186798096.
Better model found at epoch 2 with accuracy value: 0.89702969789505.
Better model found at epoch 3 with accuracy value: 0.908910870552063.
Better model found at epoch 4 with accuracy value: 0.9188118577003479.
Better model found at epoch 6 with accuracy value: 0.9227722883224487.


# Save Model (Export)

In [12]:
torch.cuda.empty_cache()
learn.load('best-trashnet-model')
#saved_model = learn.export()
#load_model = learn.load_learner()

AttributeError: 'Learner' object has no attribute 'eval'

In [ ]:
# Results
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
doc(interp.plot_top_losses)

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
# Clean uyp top-losses
from fastai.widgets import *

#ds, idxs = DatasetFormatter().from_toplosses(learn)
#ImageCleaner(ds, idxs, paths, 1)

In [ ]:
import time
defaults.device = torch.device('cuda')

start = time.time()
test = open_image('test.jpg')
pred_class,pred_idx,outputs = learn.predict(test)
pred_class
print('Took {} seconds'.format(time.time()-start))